# 导入数据

In [3]:
import pandas as pd


folder_path = "data/"

# 读取data.S1.xlsx和data.S2.xlsx文件
df_s1 = pd.read_excel(folder_path + 'data.S1_scaled.xlsx')
df_s2 = pd.read_excel(folder_path + 'data.S2_scaled.xlsx')

df = pd.concat([df_s1, df_s2], axis=0)

df = df.reset_index(drop=True)

print(df_s1.shape)
print(df_s2.shape)
print(df.shape)

(753, 36)
(88, 36)
(841, 36)


# 处理特殊列标签

In [24]:
y = df.iloc[:, -1]


# 选择第11列到第33列的数据（根据0-based索引）
element_columns = df.iloc[:, 10:33]

# 创建一个新的 DataFrame 包含这些列
new_df = pd.DataFrame(element_columns)

new_df['Nb (mass%)'] = new_df['Nb+Ta (mass%)'] / 2
new_df['Ta (mass%)'] = new_df['Nb+Ta (mass%)'] / 2

# 删除 "Nb+Ta" 列
new_df.drop(columns=['Nb+Ta (mass%)'], inplace=True)

# matminer添加描述符

In [25]:
from matminer.featurizers.conversions import StrToComposition, CompositionToOxidComposition
from matminer.featurizers.composition import ElementProperty, OxidationStates


extracted_data = pd.DataFrame()
# 遍历每一行和每一列
for row_index, row in new_df.iterrows():
    for col_name, value in row.items():
        # 使用字符串分割方法，提取列标签中括号前的字母
        label = col_name.split('(')[0]
        
        # 去除字母和数值之间的空格并连接成一个字符串
        label_value = f"{label}{value}".replace(" ", "")
        
        # 将连接后的字符串添加到提取数据的 DataFrame
        extracted_data.loc[row_index, label] = label_value

# 添加一列，将每行的内容连接成一个字符串，每一列的数据前加一个空格
extracted_data['formula'] = ""

extracted_data['formula'] = extracted_data.apply(lambda row: ' '.join([' ' + str(val) for val in row]), axis=1)




df_pec = StrToComposition().featurize_dataframe(extracted_data, "formula")
ep_feat = ElementProperty.from_preset(preset_name="magpie")
df_pec_magpie = ep_feat.featurize_dataframe(df_pec, col_id="composition") 


StrToComposition:   0%|          | 0/5 [00:00<?, ?it/s]

ElementProperty:   0%|          | 0/5 [00:00<?, ?it/s]

# 得到描述符数据和实验参数数据

In [26]:
# 删除前 26 列（成分数据），保留从第 27 列开始的列（描述符数据）
df_comp = df_pec_magpie.iloc[:, 26:]
print(df_comp.shape)

# 选取实验数据
df_exp = df.iloc[:, 1:10].join(df.iloc[:, 33:-1], how='left')
print(df_exp.shape)

(5, 132)
(5, 11)


# 分别minmaxscaler

In [28]:
from sklearn.preprocessing import MinMaxScaler

# 创建 Min-Max 缩放器
scaler = MinMaxScaler()

# 对 df_comp 进行缩放
df_comp_scaled = scaler.fit_transform(df_comp)

# 对 df_exp 进行缩放
df_exp_scaled = scaler.fit_transform(df_exp)

# # 转换为 DataFrame（可选，如果你希望继续使用 DataFrame 格式）
# df_comp_scaled = pd.DataFrame(df_comp_scaled, columns=df_comp.columns)
# df_exp_scaled = pd.DataFrame(df_exp_scaled, columns=df_exp.columns)



# 分别pca降维

In [35]:
from sklearn.decomposition import PCA

def PCA_dim_select(selected_method, n_dims):
    if selected_method == 'auto':
        assert type(n_dims) == float
        selected_dim = n_dims
    elif selected_method == 'assigned':
        # assert type(n_dims) == int
        selected_dim = int(n_dims)
    return selected_dim


# 创建 PCA 模型，指定要保留的主成分数量（你可以根据需要调整）
comp_dims = 0.98  # 选择要保留的主成分数量，必须要小于行数和列数
exp_dims = 0.95
# pca = PCA(n_components=PCA_dim_select('assigned', n_dims))

# 创建不同的PCA实例

pca_comp = PCA(n_components=PCA_dim_select('auto', comp_dims))
pca_exp = PCA(n_components=PCA_dim_select('auto', exp_dims))


# 对 df_comp 进行 PCA 降维
df_comp_pca = pca_comp.fit_transform(df_comp_scaled)

# 对 df_exp 进行 PCA 降维
df_exp_pca = pca_exp.fit_transform(df_exp_scaled)

print(df_comp_pca.shape)
print(df_exp_pca.shape)

# 列数据合并，行数据分离

In [42]:
combined_data = pd.concat([pd.DataFrame(df_comp_pca), pd.DataFrame(df_exp_pca)], axis=1)
final_data = pd.concat([combined_data, y], axis=1)

final_data = final_data.reset_index(drop=True)




# 将合并后的数据拆分回原来的两个数据库
df_s1_scaled = final_data.iloc[:len(df_s1), :]
df_s2_scaled = final_data.iloc[len(df_s1):, :]



(5, 5)
(5, 6)


In [43]:

folder_path = "data/"

# 保存处理后的 df_s1_scaled 到新的 Excel 文件
df_s1_scaled.to_excel(folder_path + 'data.S1_mat_pca.xlsx', index=False)

# 保存处理后的 df_s2_scaled 到新的 Excel 文件
df_s2_scaled.to_excel(folder_path + 'data.S2_mat_pca.xlsx', index=False)